# IMPORT REQUIRED LIBRARY

In [150]:
import pandas as pd
import os
import gensim
from snownlp import SnowNLP

# GET DATA

In [2]:
path = os.path.abspath(os.getcwd())
filename = path+"\\dataset\\organized_dataset\\csv-final.csv"

In [3]:
## Check data type
import chardet
with open(filename, 'rb') as file:
    print(chardet.detect(file.read()))

{'encoding': 'UTF-16', 'confidence': 1.0, 'language': ''}


In [4]:
df = pd.read_csv(filename, encoding = 'UTF-16', sep='\t')

In [5]:
check = df['Body'].isnull()
contents = [df['Body'][i] for i in range(len(check)) if check[i] == False] #Filter the email which has non content

In [6]:
for i in range(len(contents)):
    for j in range(len(contents[i])):
        if contents[i][j:j+len('SecuShare')] == 'SecuShare': #Cut the information for original email
            contents[i] = contents[i][:j]

In [162]:
from tqdm import tqdm
def sent_to_words(sentences):
    word_list = list()
    for i in tqdm(range(len(sentences))):
        word_list.append(SnowNLP(sentences[i]).words)
    return word_list

data_words = sent_to_words(contents)

100%|██████████████████████████████████████████████████████████████████████████████| 2525/2525 [04:12<00:00, 10.00it/s]


In [201]:
id2word[1]

'(203)'

In [205]:
import gensim.corpora as corpora
def sentiment_score(key_words):    
    corpus_dict = dict()
    
    for key_word in key_words:
        s = SnowNLP(SnowNLP(key_word).han) #Conver Traditional chinese to simplfied chinese
        if key_word in corpus_dict:
            corpus_dict[key_word] = corpus_dict[key_word] + s.sentiments
        else:
            corpus_dict[key_word] = s.sentiments
    return corpus_dict

corpus = list()
for i in tqdm(range(len(data_words))):
    corpus.append(sentiment_score(data_words[i]))

100%|██████████████████████████████████████████████████████████████████████████████| 2525/2525 [01:53<00:00, 22.23it/s]


In [223]:
id2word = corpora.Dictionary(data_words)

In [237]:
corpus_list = list()
for i in tqdm(range(len(corpus))):
    sentence_list = list()
    for key in corpus[i]:
        sentence_list.append((*id2word.doc2idx([key]) , corpus[i][key])) #append((token, score))
    corpus_list.append(sentence_list)

100%|████████████████████████████████████████████████████████████████████████████| 2525/2525 [00:01<00:00, 2209.38it/s]


In [244]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_list,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=5,
                                           alpha='auto',
                                           per_word_topics=True)

In [246]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus_list]

[(0,
  '0.027*"，" + 0.015*"的" + 0.012*"資" + 0.009*"謝" + 0.008*"有" + 0.008*"。" + '
  '0.007*"掃" + 0.007*"個" + 0.007*"我" + 0.006*"題"'),
 (1,
  '0.049*"，" + 0.028*"謝" + 0.023*"的" + 0.020*"您" + 0.018*"題" + 0.014*"有" + '
  '0.010*"。" + 0.009*"問" + 0.008*"上" + 0.007*"是"'),
 (2,
  '0.030*"，" + 0.022*"的" + 0.019*"校" + 0.018*"電子" + 0.018*"園" + 0.016*"教育" + '
  '0.014*"部" + 0.014*"雲端" + 0.014*"題" + 0.011*"您"')]


In [254]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus_list, texts=contents)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)


# Show
df_dominant_topic.head(10)

C:\Users\lemin\AppData\Local\Temp\ipykernel_14764\3569576796.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
C:\Users\lemin\AppData\Local\Temp\ipykernel_14764\3569576796.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0,0.5974,"，, 的, 資, 謝, 有, 。, 掃, 個, 我, 題",Dear 網擎：\r\n 附件所示 郵件使用者 alex.chen@cr...
1,1,1,0.7382,"，, 謝, 的, 您, 題, 有, 。, 問, 上, 是","您好，使用者反應重新註冊帳號，將訂單送出時，系統會顯示""認證失敗，請重新確認您所輸入的帳號密..."
2,2,1,0.9083,"，, 謝, 的, 您, 題, 有, 。, 問, 上, 是",您好，國民健康署張淇鈞 小姐02-2522 0889rubychang@hpa.gov.tw...
3,3,1,0.9845,"，, 謝, 的, 您, 題, 有, 。, 問, 上, 是",您好，客戶來電表示還沒收到回覆，今天上午也有寄信到m2k_noc，麻煩您幫忙盡快連繫，謝謝。...
4,4,1,0.9884,"，, 謝, 的, 您, 題, 有, 。, 問, 上, 是",您好，德律陳先生0933 818 259問題：客戶提到管理者帳號被登入，想確認系統設定是否有...
5,5,1,0.8914,"，, 謝, 的, 您, 題, 有, 。, 問, 上, 是",您好，卓墾雲端有再來電表示客戶現在無法收發信，客戶很急，麻煩您幫忙盡快連繫，謝謝。Jasmi...
6,6,1,0.9841,"，, 謝, 的, 您, 題, 有, 。, 問, 上, 是",您好，客戶來電表示還沒收到回覆，客戶很急，麻煩您幫忙盡快連繫，謝謝您。Jasmine謝謝您對...
7,7,1,0.9928,"，, 謝, 的, 您, 題, 有, 。, 問, 上, 是",您好，客戶急著知道企業信箱代管價格，台鹽綠能0910761594 賴小姐潛在客戶來電表示，想...
8,8,1,0.9875,"，, 謝, 的, 您, 題, 有, 。, 問, 上, 是",您好，王小姐急著知道 MailCloud 企業信箱代管服務價格，穩鈦興業有限公司外銷-王小姐...
9,9,1,0.9829,"，, 謝, 的, 您, 題, 有, 。, 問, 上, 是",您好，溫先生表示現在信件都進不來，麻煩您幫忙盡快連繫，謝謝您。Jasmine謝謝您對 Mai...


In [306]:
SnowNLP(contents[0]).words
s = SnowNLP(SnowNLP('您好，奇郁國際左小姐26572928#212問題：找Oni，左小姐提到今天有到公司安裝SPAM系統，有問題要再確認，麻煩您幫忙盡快連繫，謝謝您。Jasmine謝謝您對 Mail2000 的愛用。\r\n若有任何問題，歡迎再次洽詢').han) #Conver Traditional chinese to simplfied chinese
s.sentiments

1.3762801751138198e-05

In [308]:
s1 = 0
s2 = 0
s0 = 0
for i in range(2525):
    if df_dominant_topic.iloc[i]['Dominant_Topic'] == 0:
        s0 += 1
        print(i)
    elif df_dominant_topic.iloc[i]['Dominant_Topic'] == 1:
        s1 += 1
    elif df_dominant_topic.iloc[i]['Dominant_Topic'] == 2:
        s2 += 1
print(s0,'-',s1,'-',s2)

0
64
191
200
220
221
227
233
234
235
236
237
239
240
241
243
244
245
246
247
248
249
250
251
252
253
254
255
257
258
259
260
261
262
263
268
269
270
271
272
273
274
275
276
277
278
279
280
281
299
300
301
302
303
304
372
373
374
375
380
381
384
385
386
387
388
391
397
399
402
404
406
407
408
409
410
411
412
414
415
416
417
418
419
420
421
422
423
424
426
427
428
429
430
431
432
433
434
435
437
438
439
441
443
444
445
446
447
449
450
451
452
453
454
455
456
458
460
464
478
510
523
524
531
534
566
582
614
634
653
654
679
681
695
696
697
698
699
701
729
756
757
758
759
760
761
762
784
797
865
882
883
885
886
889
890
891
893
894
895
896
897
898
900
901
902
926
927
928
930
931
932
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
952
953
955
956
957
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
981
982
983
984
985
986
987
988
990
991
992
998
1006
1015
1022
1023
1024
1025
1026
1027
1028
1029
1030
1032
1033
1034
1035
1036
1037
1038
1039
1045
1051
1052
1053
1054

In [309]:
contents[523]

'您好，\xa0dpforest@mail2000.com.tw\xa0問題：請協助更換這位用戶的續約個人專屬優惠，之後他來續約的時候，請改成 399\xa0 20GB 的方案。\xa0他目前是 999 60GB，請問可以將這個容量維持到他約滿，讓他續約的時候才看到 399 GB 的選項嗎？\xa0\xa0請您協助確認，感謝。\xa0Best Regards,Kate林冠伶 Kate Lin網擎資訊軟體股份有限公司 / \r\n雲端服務Online Service Dept., Openfind Information Technology, \r\nInc.台北市大同區重慶北路二段 243 號 7 樓Email: kate_lin@openfind.com.twTEL:+886 2 \r\n25532000 ext.745網擎企業郵件代管 http://www.mailasp.com.tw/Mail2000 電子信箱 \r\nhttp://mail2000.com.tw/Openfind 網擎資訊 \r\nhttp://www.openfind.com/'